In [ ]:
import numpy as np
import pandas as pd
# from BHDVCS_tf import BHDVCStf
from BHDVCS_tf import TotalFLayer
from BHDVCS_tf import DvcsData
from BHDVCS_tf import cffs_from_globalModel
from BHDVCS_tf import F2VsPhi as F2VsPhitf
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt

import sys
from scipy.stats import chisquare

ddf=pd.read_csv(r"/content/Book 8.csv",header=0,
                usecols=["# of layers", "nodes first layer", "decreasing nodes",
                         "activation function","initial learning rate","decay rate"])
#note to call funcitons use ddf.iloc[row][column] with normal python index at 0

#for j in range(len(ddf.iloc[:,0])):
for j in range(len(ddf.iloc[:,0])):
  df = pd.read_csv(r"/content/PseudoKM15_New_FormFactor.csv", dtype=np.float64)
#first is what the column is called in csv, second is new name of the column in the data frame.
  df = df.rename(columns={"sigmaF": "errF"})

  data = DvcsData(df)

  initializer = tf.keras.initializers.HeNormal()


  kinematics = tf.keras.Input(shape=(4))

  k=0
  while k <ddf.iloc[j,0]:
    if k==0:
      x0=tf.keras.layers.Dense(ddf.iloc[j][1], activation=ddf.iloc[j][3], kernel_initializer=initializer)(kinematics)
    else:
      xo=x0
      x0=tf.keras.layers.Dense(ddf.iloc[j][1]-k*ddf.iloc[j][2], activation=ddf.iloc[j][3], kernel_initializer=initializer)(xo)
      #x0=tf.keras.layers.Dense(ddf.iloc[j][1]-k*ddf.iloc[j][2], activation=ddf.iloc[j][3], kernel_initializer=initializer)(x0) the next stepp
    k=k+1

  #x2 = tf.keras.layers.Dense(100, activation="linear", kernel_initializer=initializer)(x1)
  outputs = tf.keras.layers.Dense(4, activation="linear", kernel_initializer=initializer)(x0)
  noncffInputs = tf.keras.Input(shape=(7))
#### phi, kin1, kin2, kin3, kin4, F1, F2 ####, putting the noncff and the cff gotten from the NN together.
  total_FInputs = tf.keras.layers.concatenate([noncffInputs,outputs])
  TotalF = TotalFLayer()(total_FInputs)

# Model puts it all together, input kin and noncff separtly, but together, output the TotalF from above as output single thing
  tfModel = tf.keras.Model(inputs=[kinematics, noncffInputs], outputs = TotalF, name="tfmodel")
#early stopping jsut stops it if it is bad.
  early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0000005, patience=100)

#I assume this is the optimizers
#this is learning rate, initial learning rate, df is decay steps, decay rate.
  lr = tf.keras.optimizers.schedules.ExponentialDecay(
    ddf.iloc[j][4], df.shape[0]/1, ddf.iloc[j][5], staircase=False, name=None)

#I assume this complies it all
  tfModel.compile(optimizer = tf.keras.optimizers.Adam(lr),loss = tf.keras.losses.MeanSquaredError())

#returns a numpy array of the wheights used and the biases of each layer
  Wsave = tfModel.get_weights()

#!!High-overfitting from batch_size 1, 2 100 node hidden layers no validation data, huge number of epochs!!#
# Over-fitting to F will likely not reflect well in CFF predictions

#Number of kinematic sets
  by_set = []
  for i in range(15):
    setI = data.getSet(i, itemsInSet=45)

    tfModel.set_weights(Wsave)
#epoch is how many times it runs over the code, set to 100. take
    tfModel.fit([setI.Kinematics, setI.XnoCFF], setI.sampleY(), # one replica of samples from F vals
                epochs=1001, verbose=0, batch_size=16)

    cffs = cffs_from_globalModel(tfModel, setI.Kinematics, numHL=2)

    by_set.append(cffs)

  df = pd.DataFrame(by_set)

  if len(sys.argv) > 1:
    df.to_csv('bySetCFFs' + sys.argv[1] + str(j)+'.csv')
  else:
    df.to_csv('bySetCFFs' + str(j) +'.csv')